In [11]:
from platform import python_version
print(python_version())

3.9.16


In [1]:
# import matplotlib.pyplot as plt
import cv2
import easyocr
import openai
import speech_recognition as sr
import pyttsx3
import datetime
import random
import time

from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from threading import Thread, Event
from pyfirmata import Arduino, SERVO, util
from time import sleep
# from pylab import rcParams


In [3]:
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice',voices[0].id)
# rcParams['figure.figsize'] = 8, 16

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

event1 = Event()
event2 = Event()

In [16]:
board = Arduino('COM5')  # COM5 is the communication channel for the Arduino Uno

board.digital[4].mode = SERVO  # hre = horizontal right eye (Wire in Arduino = White , Range is (30, 90, 150)) (50, 90, 130)
board.digital[5].mode = SERVO  # hle = horizontal left eye (Wire in Arduino = Green, Range is (20, 80, 140)) (40, 80, 120)
board.digital[6].mode = SERVO  # vre = vertical right eye (Wire in Arduino = Yellow , Range is (30, 90, 150)) (50, 90, 130)
board.digital[7].mode = SERVO  # vle = vertical left eye (Wire in Arduino = Violet)Range is (30, 90, 150)) (50, 90, 130)
board.digital[8].mode = SERVO  # uel = upper eye lids (Wire in Arduino = Orange, Range is (60, 110))
board.digital[9].mode = SERVO  # lel = lower eye lids (Wire in Arduino = Brown)Range is (90, 120))
board.digital[10].mode = SERVO  # Upper Lip (Wire in Arduino = Red)


def rotate_servo(pin, angle):
    board.digital[pin].write(angle)   

    
rotate_servo(4, 90)
rotate_servo(5, 90)
rotate_servo(6, 90)
rotate_servo(7, 90)
rotate_servo(8, 90)
rotate_servo(9, 100)
rotate_servo(10, 100)

Additional Functions (Experimental)

In [5]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

vision_keywords = ["see", "scan", "read"]

In [6]:
# Variables with inputs to motors for different positions of eyeballs and eyelids

Front = [90, 90]
Left = [150, 90]
Right = [30, 90]
Top = [90, 150]
Lower = [90, 30]
Top_left = [30, 30]
Top_right = [30, 150]
Lower_left = [150, 30]
Lower_right = [30, 30]

In [7]:
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

Working Functions

In [8]:
def blinking():
    rotate_servo(8, 130) # upper Eye Lid
    rotate_servo(9, 90) # lower Eye Lid
    sleep(0.1) # 100ms
    rotate_servo(8, 90)
    rotate_servo(9, 120)
    
    
def talk(event1):
    event1.clear()
    while True:
        rotate_servo(10, 100)
        sleep(0.1)
        rotate_servo(10, 70)
        sleep(0.1)
        rotate_servo(10, 100)
        
        sleep(random.triangular(0, 0.5))
        
        if event1.is_set():
            break
        
    
def idle_looking(event2):
    event2.clear()
    while True:
        a = random.randint(50, 130)
        b = random.randint(50, 130)

        # Eyeballs code
        rotate_servo(4, a)
        rotate_servo(5, a)
        rotate_servo(6, b)
        rotate_servo(7, 180-b)
        if a%2 == 0:
            blinking()
        sleep(random.randint(0, 3))
        
        if event2.is_set():
            break

In [9]:
def wish_master():
    hour = int(datetime.datetime.now().hour)
    if hour>=0 and hour<12:
        engine_speak("Good Morning!")
    elif hour>=12 and hour<18:
        engine_speak("Good Afternoon!")   
    else:
        engine_speak("Good Evening!")
    engine_speak("I am decypher. Sir, how may I help you")
    

def engine_speak(audio):
    Thread(target = talk, args=(event1,)).start()
    print(audio)
    engine.say(audio)
    engine.runAndWait()
    event1.set()
    
    
    
def engine_listen():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        r.dynamic_energy_threshold = True
        r.pause_threshold = 1
        r.operation_timeout = 1
        print('\n')
        print("Listening...")
        audio = r.listen(source)
        
    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language = "en-US").lower()
        print(f"User said: {query}")
        return query
    
    except:
        engine_speak("Please repeat that one")
        engine_listen()

In [10]:
def camera():
    cap = cv2.VideoCapture(0)
    
    # Check if camera opened successfully
    if (cap.isOpened()== False):
        print("Error opening video file")
    frame_no = 0
    # Read until video is completed
    while(cap.isOpened()):
        frame_no += 1
    # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
        # Display the resulting frame
            cv2.imshow('Video', frame)
            key = cv2.waitKey(1)
            if frame_no == 150:
                image_name = "image.png"
                cv2.imwrite(image_name, frame)
                break
  
    # When everything is done, release the video capture object
    cap.release()
  
    # Closes all the frames
    cv2.destroyAllWindows()

In [11]:
def textextractor():
    reader = easyocr.Reader(["en"], gpu = False)
    output = reader.readtext("image.png")
    prompt = ""
    for i in range(len(output)):
        prompt = prompt + output[i][1] + " "
        prompt = str(prompt)
        
    return prompt

In [12]:
def chatbot(query):
    openai.api_key = 'API Key'
    model_engine = 'text-davinci-003'
    prompt = str(query)

    completion = openai.Completion.create(
        engine = model_engine,
        prompt = prompt,
        max_tokens = 1024,
        n = 1,
        stop = None,
        temperature = 0.5,
    )

    response = completion.choices[0].text
    return response

In [13]:
def ImageCaptioning():
    camera()
    image = Image.open("image.png").convert('RGB')
    
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    
    return(processor.decode(out[0], skip_special_tokens=True))

In [14]:
def Decypher():
    Thread(target = idle_looking, args=(event2,)).start()
    wish_master()
    
    while True:
        query = engine_listen()
        
        if query is not None:
            if "read" in query or "written" in query:
                response = camera()
                response = textextractor()
                print(response)
                # jhkjkjh
                engine_speak("Yes I can sir.")
                engine_speak(response)
            
            elif "see" in query or "scan" in query:
                response = ImageCaptioning()
                engine_speak(response)
                 
            elif "bye robot" in query or "shutdown robot" in query:
                engine_speak("It was a pleasure to meet you, have a nice day sir.")
                break

            else:
                response = chatbot(query)
                engine_speak(response)
                
    event2.set()
    rotate_servo(4, 90)
    rotate_servo(5, 90)
    rotate_servo(6, 90)
    rotate_servo(7, 90)
    rotate_servo(8, 90)
    rotate_servo(9, 100)
    rotate_servo(10, 100)

In [ ]:
Decypher()

Good Morning!
I am decypher. Sir, how may I help you


Listening...
